In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import scipy  # Yêu cầu scipy để sử dụng bộ dữ liệu Flowers102

# Tải Flower dataset tự động từ torchvision
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Tải bộ dữ liệu Flowers từ torchvision.datasets
from torchvision.datasets import Flowers102

train_dataset = Flowers102(root='./data', split='train', download=True, transform=transform)
test_dataset = Flowers102(root='./data', split='test', download=True, transform=transform)

train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Xây dựng mô hình CNN cho nhận diện tên các loại hoa
class FlowerCNN(nn.Module):
    def __init__(self, num_classes):
        super(FlowerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Khởi tạo mô hình
num_classes = len(train_dataset._labels)
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = FlowerCNN(num_classes).to(device)

# Định nghĩa loss và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện mô hình
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Đặt lại gradient về 0
        optimizer.zero_grad()

        # Forward và backward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Đánh giá mô hình
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test accuracy: {100 * correct / total:.2f}%')

# Lưu mô hình
torch.save(model.state_dict(), 'flower_cnn_model.pth')


Epoch [1/10], Loss: 5.4289
Epoch [2/10], Loss: 4.2810
Epoch [3/10], Loss: 3.6780
Epoch [4/10], Loss: 3.1123
Epoch [5/10], Loss: 2.4451
Epoch [6/10], Loss: 1.8179
Epoch [7/10], Loss: 1.1682
Epoch [8/10], Loss: 0.7050
Epoch [9/10], Loss: 0.3984
Epoch [10/10], Loss: 0.2622
Test accuracy: 18.18%


FileNotFoundError: [Errno 2] No such file or directory: '/Users/levanduy/PycharmProjects/BaiNhom3/path/to/your/flower_image.jpg'